In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
import os
from dotenv import load_dotenv
from src.etl import load as lo #con jupyter
import unicodedata

In [2]:
load_dotenv()

True

In [3]:
#Llamamos a la variable de entorno
DB_NAME = os.getenv("DB_NAME")
DB_USER=os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST= os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

In [4]:
conn, cur = lo.crear_conexión()

In [5]:
#Comprobamos que la conexión está bien realizada:
cur.execute("SELECT version();")
cur.fetchone() #para que muestre el resultado de la query anterior

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

# Creación de los datos para las tablas:

In [6]:
#Importo los df procesados:
ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS')
ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS =os.getenv('ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS')
ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS')
ARCHIVO_GUARDAR_DATOS_API_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_DATOS_API_PROCESADOS')
ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE=os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE')

## 1. Tabla pais_destino

###  1.1 Preparación de los datos: Unión de los paises y continentes tanto escrapeados como obtenidos de la API de turismo emisor

#### a) Turismo emisor

In [7]:
df_turismo_emisor_procesado=pd.read_csv(ARCHIVO_GUARDAR_DATOS_API_PROCESADOS)
df_turismo_emisor_procesado.columns

Index(['Unnamed: 0', 'AÑO', 'MES', 'CCAA_ORIGEN', 'CONTINENTE_DESTINO',
       'PAIS_DESTINO', 'TURISTAS', 'PERNOCTACIONES', 'ESTANCIA_MEDIA'],
      dtype='object')

In [8]:
df_turismo_emisor_procesado.head()

,Unnamed: 0,AÑO,MES,CCAA_ORIGEN,CONTINENTE_DESTINO,PAIS_DESTINO,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
0,0,2019,7,Andalucia,America,Argentina,"419,0","3616,0","8,6"
1,1,2019,7,Andalucia,America,Bolivia,"61,0","486,0","8,0"
2,2,2019,7,Andalucia,America,Brasil,"277,0","2214,0","8,0"
3,3,2019,7,Andalucia,America,Canada,"483,0","6159,0","12,8"
4,4,2019,7,Andalucia,America,Chile,"242,0","2068,0","8,5"


In [9]:
df_paises_continentes_turismos_emisor=df_turismo_emisor_procesado[['PAIS_DESTINO','CONTINENTE_DESTINO']]
df_paises_continentes_turismos_emisor=df_paises_continentes_turismos_emisor.drop_duplicates()
df_paises_continentes_turismos_emisor.CONTINENTE_DESTINO.unique()

array(['America', 'Asia', 'Europa', 'Oceania', 'Africa'], dtype=object)

In [10]:
df_paises_continentes_turismos_emisor.columns

Index(['PAIS_DESTINO', 'CONTINENTE_DESTINO'], dtype='object')

In [11]:
df_paises_continentes_turismos_emisor= df_paises_continentes_turismos_emisor.rename(
    columns={'PAIS_DESTINO':'nombre_pais_destino', 'CONTINENTE_DESTINO':'nombre_continente'})
df_paises_continentes_turismos_emisor= df_paises_continentes_turismos_emisor.sort_values(by='nombre_pais_destino').reset_index(drop=True)

In [12]:
df_paises_continentes_turismos_emisor

,nombre_pais_destino,nombre_continente
0,Albania,Europa
1,Alemania,Europa
2,Andorra,Europa
3,Angola,Africa
4,Antigua,America
...,...,...
153,Venezuela,America
154,Vietnam,Asia
155,Yemen,Asia
156,Zambia,Africa


#### b) Paises escrapeados

In [13]:
df_paises_continentes_escrapeo = pd.read_pickle(ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS)
df_paises_continentes_escrapeo.columns

Index(['continente', 'pais', 'fecha_escrapeo', 'en_ultimo_escrapeo'], dtype='object')

In [14]:
df_paises_continentes_escrapeo.head()

,continente,pais,fecha_escrapeo,en_ultimo_escrapeo
0,Europa,Albania,2025-04-14,Si
1,Europa,Alemania,2025-04-14,Si
2,America,Antigua,2025-04-14,Si
3,America,Argentina,2025-04-14,Si
4,America,Aruba,2025-04-14,Si


In [15]:
df_paises_continentes_escrapeo=df_paises_continentes_escrapeo[['pais','continente']]
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.drop_duplicates()

In [16]:
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.rename(
    columns={'pais':'nombre_pais_destino', 'continente':'nombre_continente'})

In [17]:
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.sort_values(by='nombre_pais_destino').reset_index(drop=True)
df_paises_continentes_escrapeo

,nombre_pais_destino,nombre_continente
0,Albania,Europa
1,Alemania,Europa
2,Antigua,America
3,Arabia Saudi,Asia
4,Argentina,America
...,...,...
120,Uruguay,America
121,Uzbekistan,Asia
122,Vietnam,Asia
123,Zanzibar,Africa


#### c) Paises en itinerarios ciudades

In [18]:
df_itinerarios_ciudades = pd.read_pickle(ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS)
df_itinerarios_ciudades.columns

Index(['itinerario_modificado_para_dividir', 'ciudad', 'pais_correcto'], dtype='object')

In [19]:
df_itinerarios_ciudades_pais_continentes = pd.DataFrame()
df_itinerarios_ciudades_pais_continentes['pais'] = df_itinerarios_ciudades['pais_correcto'].drop_duplicates().reset_index(drop=True)

In [20]:
df_itinerarios_ciudades_pais_continentes

,pais
0,Alemania
1,Argentina
2,Aruba
3,Australia
4,Austria
...,...
123,Macedonia Del Norte
124,Bosnia Y Herzegovina
125,Belice
126,Nigeria


In [21]:
df_opencage = pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE)
df_opencage.columns

Index(['pais', 'continente_api', 'pais_api', 'latitud', 'longitud', 'ciudad'], dtype='object')

In [22]:
dict_paises_continentes_ciudades = {'pais':[], 'continente':[]}
for pais in df_itinerarios_ciudades_pais_continentes.pais:
    if pais in df_paises_continentes_escrapeo.nombre_pais_destino.tolist():
        pass
    else:
        if pais in df_paises_continentes_turismos_emisor.nombre_pais_destino.tolist():
           pass
        else:
           if pais in df_opencage.pais_api.tolist():
            #print(f'{pais} ya está en continentes opencage')
            dict_paises_continentes_ciudades['pais'].append(pais)
            continente = (df_opencage[df_opencage.pais_api == pais]).continente_api.unique()[0]
            dict_paises_continentes_ciudades['continente'].append(continente)
           else:
                print(f'{pais} ERROR')
df_paises_continentes_ciudades = pd.DataFrame(dict_paises_continentes_ciudades)
df_paises_continentes_ciudades

Informacion Desconocida ERROR


,pais,continente
0,Chequia,Europa


In [23]:
df_paises_continentes_ciudades= df_paises_continentes_ciudades.rename(
    columns={'pais':'nombre_pais_destino', 'continente':'nombre_continente'})

In [24]:
df_paises_continentes_ciudades

,nombre_pais_destino,nombre_continente
0,Chequia,Europa


#### d) Uno los 3 DF para conseguir una única tabla con todos los paises y continentes

In [25]:
df_paises_continentes_totales=pd.concat(
    [df_paises_continentes_turismos_emisor,df_paises_continentes_escrapeo, df_paises_continentes_ciudades], axis=0)
df_paises_continentes_totales = df_paises_continentes_totales.drop_duplicates(['nombre_pais_destino', 'nombre_continente']).reset_index(drop=True)
df_paises_continentes_totales



,nombre_pais_destino,nombre_continente
0,Albania,Europa
1,Alemania,Europa
2,Andorra,Europa
3,Angola,Africa
4,Antigua,America
...,...,...
183,Taiwan,Asia
184,Turks And Caicos,America
185,Turquia,Asia
186,Zanzibar,Africa


In [26]:
#reviso los paises que aparecen asociados a más de un continente:
df_contador_paises_duplicados = pd.DataFrame(df_paises_continentes_totales.nombre_pais_destino.value_counts()).reset_index()
df_contador_paises_duplicados =df_contador_paises_duplicados[df_contador_paises_duplicados['count']>1]
df_contador_paises_duplicados

,nombre_pais_destino,count
0,Armenia,2
1,Turquia,2
2,Georgia,2


In [27]:
lista_paises_duplicados = df_contador_paises_duplicados.nombre_pais_destino.tolist()
lista_paises_duplicados

['Armenia', 'Turquia', 'Georgia']

In [28]:
#Elimino los indices incorrectos:
lista_indices_eliminar = []
for indice,pais in enumerate(df_paises_continentes_totales.nombre_pais_destino):
    if pais in lista_paises_duplicados:
        df_paises_continentes_totales.loc[indice, "nombre_continente"]  = 'Asia'

df_paises_continentes_totales = df_paises_continentes_totales.drop_duplicates()

In [29]:
df_paises_continentes_totales.nombre_pais_destino.value_counts()

nombre_pais_destino
Albania             1
Alemania            1
Andorra             1
Angola              1
Antigua             1
                   ..
Santa Lucia         1
Taiwan              1
Turks And Caicos    1
Zanzibar            1
Chequia             1
Name: count, Length: 185, dtype: int64

## 1.2 Carga en BBDD

La tabla pais_destino tiene que tener los siguientes campos:
- id_pais_destino SERIAL PRIMARY KEY,
- nombre_pais_destino TEXT UNIQUE,
- nombre_continente TEXT

In [37]:
#Extraigo la información de la BBDD
query_extraccion = "SELECT nombre_pais_destino, id_pais_destino FROM pais_destino"
pais_destino_dict=lo.extraer_datos_de_BBDD(query_extraccion)

In [39]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in df_paises_continentes_totales.iterrows():
    nombre_pais_destino = row["nombre_pais_destino"]
    if nombre_pais_destino in  pais_destino_dict.keys():
        pass
    else: 
        nombre_continente = row["nombre_continente"]
        data_to_insert.append([nombre_pais_destino, nombre_continente])

In [46]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO pais_destino(nombre_pais_destino, nombre_continente)
VALUES (%s, %s)
"""
lo.insertar_datos_en_BBDD(insert_query, data_to_insert)

print(f'Se han añadido {len(data_to_insert)} paises en la tabla pais_destino')

TypeError: not all arguments converted during string formatting

In [ ]:
#conn.rollback()

## 2. Tabla itinerario

### 2.1 Extraigo la información del df_itinerario_ciudades y me quedo con los itinerarios sin duplicados:

In [71]:
df_itinerarios_ciudades = pd.read_pickle(ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS)
df_itinerarios_ciudades.head()

,itinerario_modificado_para_dividir,ciudad,pais_correcto
0,Berlin,Berlin,Alemania
1,"Hamburgo, Stade, Bremen, Luneburg, Schwerin, W...",Hamburgo,Alemania
1,"Hamburgo, Stade, Bremen, Luneburg, Schwerin, W...",Stade,Alemania
1,"Hamburgo, Stade, Bremen, Luneburg, Schwerin, W...",Bremen,Alemania
1,"Hamburgo, Stade, Bremen, Luneburg, Schwerin, W...",Luneburg,Alemania


In [72]:
lista_itinerarios_unicos = df_itinerarios_ciudades.itinerario_modificado_para_dividir.sort_values().unique()
len(lista_itinerarios_unicos)

1131

### 2.2 Hago la carga en la BBDD:

La tabla itinerario tiene que tener los siguientes campos:
- id_itinerario SERIAL PRIMARY KEY
- detalle_itinerario TEXT UNIQUE

In [73]:
#Extraigo la información de la BBDD
query_extraccion = "SELECT detalle_itinerario, id_itinerario FROM itinerario"
itinerario_dict=lo.extraer_datos_de_BBDD(query_extraccion)

In [75]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for itinerario in lista_itinerarios_unicos:
    if itinerario in itinerario_dict.keys():
        pass
    else: 
        data_to_insert.append((itinerario,))  

In [77]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO itinerario(detalle_itinerario)
VALUES (%s)
"""
lo.insertar_datos_en_BBDD(insert_query, data_to_insert)

print(f'Se han añadido {len(data_to_insert)} itinerarios nuevos en la tabla pais_destino')

Se han añadido 0 itinerarios nuevos en la tabla pais_destino


## 3. Tabla ciudad

### 3.1 Extraigo la información del df_itinerario_ciudades y me quedo con las ciudades sin duplicados:

In [78]:
df_itinerarios_ciudades = pd.read_pickle(ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS)
df_itinerarios_ciudades.head()

,itinerario_modificado_para_dividir,ciudad,pais_correcto
0,Berlin,Berlin,Alemania
1,"Hamburgo, Stade, Bremen, Luneburg, Schwerin, W...",Hamburgo,Alemania
1,"Hamburgo, Stade, Bremen, Luneburg, Schwerin, W...",Stade,Alemania
1,"Hamburgo, Stade, Bremen, Luneburg, Schwerin, W...",Bremen,Alemania
1,"Hamburgo, Stade, Bremen, Luneburg, Schwerin, W...",Luneburg,Alemania


In [88]:
df_itinerarios_ciudades.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9282 entries, 0 to 3346
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   itinerario_modificado_para_dividir  9282 non-null   object
 1   ciudad                              9282 non-null   object
 2   pais_correcto                       9282 non-null   object
dtypes: object(3)
memory usage: 290.1+ KB


In [79]:
lista_ciudades_unicas = df_itinerarios_ciudades.ciudad.sort_values().unique()
len(lista_ciudades_unicas)

2040

In [82]:
#creo el DF con la ciudad el pais para preparar los datos para la carga y elimino duplicados:
df_ciudad_pais_unicos = df_itinerarios_ciudades[['ciudad','pais_correcto']].drop_duplicates().reset_index(drop=True)
df_ciudad_pais_unicos

,ciudad,pais_correcto
0,Berlin,Alemania
1,Hamburgo,Alemania
2,Stade,Alemania
3,Bremen,Alemania
4,Luneburg,Alemania
...,...,...
2098,Dilijan,Armenia
2099,Sanahin Tbilisi,Georgia
2100,Baku,Azerbaiyan
2101,Gobustan,Azerbaiyan


In [86]:
#Comprueba las ciudades duplicadas
ciudades_duplicadas = []
for ciudad in df_ciudad_pais_unicos.ciudad.unique():
    df_filtrado = df_ciudad_pais_unicos[df_ciudad_pais_unicos.ciudad == ciudad]
    if len(df_filtrado.pais_correcto.unique()) >1:
        ciudades_duplicadas.append(ciudad)
        if len(df_filtrado.pais_correcto.unique()) ==2:
            pais1= df_filtrado.pais_correcto.unique()[-2]
            pais2=df_filtrado.pais_correcto.unique()[-1]
            print(f'La ciudad {ciudad} aparece en los paises {pais1}, {pais2}')
        elif len(df_filtrado.pais_correcto.unique()) ==3:
            pais3= df_filtrado.pais_correcto.unique()[-3]
            pais2= df_filtrado.pais_correcto.unique()[-2]
            pais3=df_filtrado.pais_correcto.unique()[-1]
            print(f'La ciudad {ciudad} aparece en los paises {pais1}, {pais2}, {pais3}')
        else: 
            pais4= df_filtrado.pais_correcto.unique()[-4]
            pais3= df_filtrado.pais_correcto.unique()[-3]
            pais2= df_filtrado.pais_correcto.unique()[-2]
            pais3=df_filtrado.pais_correcto.unique()[-1]
            print(f'La ciudad {ciudad} aparece en los paises {pais1}, {pais2}, {pais3} , {pais4}')


La ciudad Iguazu aparece en los paises Argentina, Brasil
La ciudad La Paz aparece en los paises Bolivia, Mexico
La ciudad Lago Titicaca aparece en los paises Bolivia, Peru
La ciudad Sofia aparece en los paises Bulgaria, Rumania
La ciudad Monasterio De Rila aparece en los paises Bulgaria, Rumania
La ciudad Plovdiv aparece en los paises Bulgaria, Rumania
La ciudad Veliko Tarnovo aparece en los paises Bulgaria, Rumania
La ciudad Siem Reap aparece en los paises Bulgaria, Tailandia, Vietnam
La ciudad Montreal aparece en los paises Canada, Estados Unidos
La ciudad Quebec aparece en los paises Canada, Estados Unidos
La ciudad Ottawa aparece en los paises Canada, Estados Unidos
La ciudad Niagara aparece en los paises Canada, Estados Unidos
La ciudad Toronto aparece en los paises Canada, Estados Unidos
La ciudad Mil Islas aparece en los paises Canada, Estados Unidos
La ciudad Victoria aparece en los paises Canada, Malta
La ciudad Hong Kong aparece en los paises China, Taiwan
La ciudad Orlando a

In [87]:
ciudades_duplicadas

['Iguazu',
 'La Paz',
 'Lago Titicaca',
 'Sofia',
 'Monasterio De Rila',
 'Plovdiv',
 'Veliko Tarnovo',
 'Siem Reap',
 'Montreal',
 'Quebec',
 'Ottawa',
 'Niagara',
 'Toronto',
 'Mil Islas',
 'Victoria',
 'Hong Kong',
 'Orlando',
 'Nueva York',
 '',
 'St Louis',
 'Niagara Falls',
 'Punta Cana',
 'Los Cabos',
 'Fiji',
 'Kusadasi',
 'Antigua',
 'Tierras Altas',
 'Maldivas',
 'Medina',
 'Rabat',
 'Mauricio',
 'Katmandu',
 'Manchester',
 'Praga',
 'Karlovy Vary',
 'Santa Lucia',
 'Ciudad Del Cabo',
 'Phuket',
 'Koh Samui',
 'Pn Bwindi',
 'Riviera Maya',
 'Bali',
 'Zanzibar',
 'Seychelles',
 'Cataratas Victoria',
 'Mozambique']

### 3.2 Hago la carga en la BBDD:

La tabla ciudad tiene que tener los siguientes campos:
- id_ciudad SERIAL PRIMARY KEY,
- nombre_ciudad TEXT UNIQUE,
- id_pais_destino INT REFERENCES pais_destino(id_pais_destino) ON DELETE CASCADE

In [ ]:
#Extraigo el id_pais_destino de la tabla pais_destino de la BBDD:
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
ciudad_dict=dict(cur.fetchall())
ciudad_dict